# Import Libraries

In [1]:
import os
import pickle
import torch
import torch.nn as nn
import pandas as pd
import importlib
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

# All imports are done relative to the 
# root of the project.
#
project_root = '../../'
if 'change_directory_to_root' not in globals():
    change_directory_to_root = True
    os.chdir(project_root)

# Imports own modules.
#
import scripts.case_study.OptimizeBESS as OptimizeBESS
import scripts.Utils as utils
import scripts.Simulation_config as Simulation_config
from scripts.Simulation_config import *


# Import Price Signal

In [2]:
# Import energy price in €/Wh
#
with open('data/exaa_prices_1h.pkl', 'rb') as f:
    price_signal = pickle.load(f)


# Import last Load Forecasts

In [3]:
# Import electrical load forecast in W
#
importlib.reload(utils)
importlib.reload(Simulation_config)

########
# Setup the test setup configuration
#
resuts_filename = 'scripts/outputs/all_train_histories.pkl'
expected_configs = [
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._1_HOUSEHOLD, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._2_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._10_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._50_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._100_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    ]
########

# Get the stored results file from the last test run
result_dict = utils.Evaluate_Models.print_results(resuts_filename, print_style = 'predicted_profiles')

# Check, if all data are available
# and create a nested dictionary of shape 'profiles_by_community_size[community_size][model][community_id]'
#
profiles_by_community_size = defaultdict(dict)
for expected_config in expected_configs:
    for available_config in result_dict:
        if expected_config == available_config:
            community_size = expected_config.aggregation_Count[0]
            profiles_by_community_size[community_size] = result_dict[expected_config]
assert len(profiles_by_community_size) == len(expected_configs), \
    f"Not all expected test-runs found: {len(profiles_by_community_size)} != {len(expected_configs)}."


## MILP Optimization

In [6]:
importlib.reload(OptimizeBESS)

# Run the MILP optimziation for all communities and model sizes.
#
costs_per_community_size = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

for community_size, profiles_per_model in profiles_by_community_size.items():
    for model_name, profiles_per_community in profiles_per_model.items():        
        
        print(f'\nOptimize community size {community_size} with model "{model_name}"')
        
        if model_name == 'Perfect':
            buffer_size = 0.0
        else:
            buffer_size = 0.15
        
        for community_id, profile in enumerate(profiles_per_community):
            perfect_prediction = profiles_per_model['Perfect'][community_id]
            myOptimization = OptimizeBESS.OptimizeBESS( profile, 
                                                        community_size, 
                                                        price_signal, 
                                                        perfect_prediction,
                                                        buffer_size = buffer_size
                                                        )
            print(f' {community_id}', end='')
            optimization_result = myOptimization.run()
            costs_per_community_size[community_size][model_name][community_id] = optimization_result



Optimize community size 1 with model "Perfect"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 1 with model "KNN"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 1 with model "PersistencePrediction"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 1 with model "xLSTM"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 1 with model "LSTM"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 1 with model "Transformer_Encoder_Only"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 2 with model "Perfect"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 2 with model "KNN"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 2 with model "PersistencePrediction"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Optimize community size 2 with model "xLSTM"
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 1

# Evaluation

In [7]:
# Calc the normalized mean absolut error:
#
def calc_nMAE(Y_perfect, Y_model):
    loss_fn = nn.L1Loss()
    Y_perfect = torch.Tensor(Y_perfect)
    Y_model = torch.Tensor(Y_model)
    loss = loss_fn(Y_perfect, Y_model)
    reference = float(torch.mean(Y_perfect))
    nMAE = 100.0 * loss / reference
    return float(nMAE)

In [ ]:
# Print all Results in a Latex Table
#
results = []
for nr_of_households, costs_per_model in costs_per_community_size.items():
        
    # Calculate the average costs without optimization
    reference_costs = []
    perfect_profiles = profiles_by_community_size[nr_of_households]['Perfect']
    for perfect_profile in perfect_profiles:
        assert price_signal.shape == perfect_profile.shape, "Arrays must be the same shape"
        assert price_signal.ndim == 1, "price_signal must be a 1D array"
        costs = np.sum(price_signal * perfect_profile)
        reference_costs.append(costs)
    avg_reference_costs = np.mean(reference_costs)
    
    if nr_of_households == 1:
        column_name = f'{nr_of_households} Household'
    else:
        column_name = f'{nr_of_households} Households'        
    
    # Store the result
    results.append({
            "Community Size": column_name,
            "Model": 'Unoptimized',
            "nMAE (%)": '',
            "Costs (€)": round(avg_reference_costs, 2),
            "Savings (€)": '',
            "Savings (%)": ''
        })
    
    for model_name, costs_per_community in costs_per_model.items():

        # Calc the average costs with optimization per model over all communities
        all_costs = list(costs_per_community.values())
        costs_avg = np.mean(all_costs)
        savings_avg = np.mean([(avg_reference_costs - costs) for costs in all_costs])
        savings_percent_avg = np.mean([100*(avg_reference_costs - costs)/avg_reference_costs for costs in all_costs])
        profile_per_community = profiles_by_community_size[nr_of_households][model_name]
        nMAE_avg = np.mean([calc_nMAE(perfect_profiles[i], profile)
                            for i, profile in enumerate(profile_per_community)])

        # Optionally rename model for printing
        if model_name == 'Transformer_Encoder_Only':
            model_name = 'Transformer'
        elif model_name == 'PersistencePrediction':
            model_name = 'Persistence'

        results.append({
            "Community Size": column_name,
            "Model": model_name,
            "nMAE (%)": round(nMAE_avg, 2),
            "Costs (€)": round(costs_avg, 2),
            "Savings (€)": round(savings_avg, 2),
            "Savings (%)": round(savings_percent_avg, 2)
        })

# Print the dataframe in latex
#
df = pd.DataFrame(results)
df["Community Size"] = df["Community Size"].mask(df["Community Size"].duplicated(), "")
latex = df.to_latex(index=False, float_format="%.2f")
lines = latex.splitlines()
new_lines = []
row_counter = 0
for i, line in enumerate(lines):
    new_lines.append(line)
    if line.endswith(r"\\") and i > 3 and not line.strip().startswith(r"\midrule"):
        row_counter += 1
        if row_counter % 7 == 0:
            new_lines.append(r"\midrule")
latex_modified = "\n".join(new_lines)
print(latex_modified)


\begin{tabular}{lllrll}
\toprule
Community Size & Model & nMAE (%) & Costs (€) & Savings (€) & Savings (%) \\
\midrule
1 Household & Unoptimized &  & 299.06 &  &  \\
 & Perfect & 0.00 & 280.65 & 18.41 & 6.16 \\
 & KNN & 53.84 & 292.77 & 6.29 & 2.10 \\
 & Persistence & 57.25 & 294.76 & 4.30 & 1.44 \\
 & xLSTM & 49.38 & 290.48 & 8.58 & 2.87 \\
 & LSTM & 49.28 & 290.45 & 8.61 & 2.88 \\
 & Transformer & 50.80 & 290.38 & 8.68 & 2.90 \\
\midrule
2 Households & Unoptimized &  & 495.71 &  &  \\
 & Perfect & 0.00 & 456.94 & 38.78 & 7.82 \\
 & KNN & 39.04 & 474.73 & 20.98 & 4.23 \\
 & Persistence & 44.98 & 478.09 & 17.62 & 3.55 \\
 & xLSTM & 37.52 & 472.74 & 22.97 & 4.63 \\
 & LSTM & 36.70 & 472.45 & 23.26 & 4.69 \\
 & Transformer & 36.92 & 472.80 & 22.91 & 4.62 \\
\midrule
10 Households & Unoptimized &  & 2218.93 &  &  \\
 & Perfect & 0.00 & 2019.64 & 199.28 & 8.98 \\
 & KNN & 21.63 & 2058.95 & 159.97 & 7.21 \\
 & Persistence & 24.86 & 2070.55 & 148.38 & 6.69 \\
 & xLSTM & 21.44 & 2060.28 & 158